In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [3]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=sk.preprocessing.normalize(train.iloc[:,1:785].copy().as_matrix())

In [4]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=sk.preprocessing.normalize(val.iloc[:,1:785].copy().as_matrix())

clf = LinearDiscriminantAnalysis()
clf.fit(training_data, training_data_class)


training_data=clf.transform(training_data)
validating_data=clf.transform(validating_data)

In [540]:
print training_data.shape

(55000, 784)


In [5]:
def create_output(class_labels):
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    num= np.exp(vector)
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]

In [6]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2):
    return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])
def dumpModel(model):
    kp=0
    for i,j in zip(model.weights,model.biases):
        np.save('./temp/weights_'+str(kp),i)
        np.save('./temp/biases_'+str(kp),j)
        kp+=1


In [135]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
        elif activation=='tanh':
            self.activation=tanh
        else:
            print 'Error : activation function not found'
                
        
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        a_set.append(a)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        
        daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*sigmoid_dif(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-dWeights[i]
            self.biases[i]=self.biases[i]-dBiases[i]

    
    
    
    def __train(self):
        global TRAINER
        oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            
        for i in range(self.max_iter):
            if i%5==0 and i!=0 and self.anneal==True:
                self.lr/=2
            
            accuracy=0
            if self.batch_size==1:
                self.batch_size=len(training_data)
            
            sets=len(training_data)/self.batch_size # number of batches
                
            for j in range(sets):
                #print 'SET=',j
                los=0
                dWeights,dBiases=paraCopy(self.weights,self.biases,0)
                for TRAINER in range(j*self.batch_size,(j+1)*self.batch_size):
                    #print 'TRAINER=',TRAINER
                    h,a,y=self.__forward_propagation()

                    
                    dWeights,dBiases=paraAdd(self.__back_propagation(h,a,y),
                                         paraCopy(dWeights,dBiases,1))
                    los+=loss(y,self.output_data[TRAINER])

                dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                         paraCopy(dWeights,dBiases,self.lr))

                self.__update(dWeights,dBiases)


                
                oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                #print np.argmax(y),self.raw_class_labels[TRAINER]
                #if np.argmax(y)==self.raw_class_labels[TRAINER]:

                 #   accuracy+=1
                print 'Epoch : ',i,'Step : ',j,'loss : ',los
                #print 'acc=',accuracy*1.0/sets

    def fit(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1],self.layers[i])))#-np.random.rand(self.layers[i+1],self.layers[i]))
            self.biases.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1])))#-np.random.rand(self.layers[i+1]))
            self.weights.append(np.random.uniform(self.layers[i+1],self.layers[i]))
            self.biases.append(np.random.rand(self.layers[i+1]))
            
            #print self.biases[-1].shape
        self.__train()
        
    def resume(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.load('./temp/weights_'+str(i)+'.npy'))
            self.biases.append(np.load('./temp/biases_'+str(i)+'.npy'))
            #print self.weights[-1].shape
        self.__train()
        
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
    def buildModel(self):
        self.weights=[]
        self.biases=[]
        for i in range(self.num_layers-1):
            self.weights.append(np.load('./temp/weights_'+str(i)+'.npy'))
            self.biases.append(np.load('./temp/biases_'+str(i)+'.npy'))
        
    

In [136]:
model=NN_Model(hidden_layers=(256,128),lr=0.001,max_iter=20,momentum=0.5,batch_size=250,anneal=True)
training_data=np.array(training_data)

In [ ]:
model.resume(training_data,training_data_class)
#model.buildModel()

Epoch :  0 Step :  0 loss :  192.056503593
Epoch :  0 Step :  1 loss :  192.194697538
Epoch :  0 Step :  2 loss :  189.293435517
Epoch :  0 Step :  3 loss :  174.903088288
Epoch :  0 Step :  4 loss :  224.038951329
Epoch :  0 Step :  5 loss :  189.176430017
Epoch :  0 Step :  6 loss :  155.872615207
Epoch :  0 Step :  7 loss :  189.827569138
Epoch :  0 Step :  8 loss :  204.811475401
Epoch :  0 Step :  9 loss :  185.680358012
Epoch :  0 Step :  10 loss :  182.822441772
Epoch :  0 Step :  11 loss :  205.427422319
Epoch :  0 Step :  12 loss :  208.247745515
Epoch :  0 Step :  13 loss :  237.490697126
Epoch :  0 Step :  14 loss :  191.22601587
Epoch :  0 Step :  15 loss :  244.966084589
Epoch :  0 Step :  16 loss :  190.349199208
Epoch :  0 Step :  17 loss :  173.581924491
Epoch :  0 Step :  18 loss :  149.374947267
Epoch :  0 Step :  19 loss :  160.047856655
Epoch :  0 Step :  20 loss :  226.194983091
Epoch :  0 Step :  21 loss :  248.755235151
Epoch :  0 Step :  22 loss :  214.658625657

Epoch :  0 Step :  185 loss :  179.204658652
Epoch :  0 Step :  186 loss :  169.957110906
Epoch :  0 Step :  187 loss :  214.509638197
Epoch :  0 Step :  188 loss :  201.586132352
Epoch :  0 Step :  189 loss :  193.588862846
Epoch :  0 Step :  190 loss :  214.287154274
Epoch :  0 Step :  191 loss :  145.960627337
Epoch :  0 Step :  192 loss :  150.587072945
Epoch :  0 Step :  193 loss :  174.539885951
Epoch :  0 Step :  194 loss :  192.608617598
Epoch :  0 Step :  195 loss :  168.797616848
Epoch :  0 Step :  196 loss :  170.947157354
Epoch :  0 Step :  197 loss :  186.915124958
Epoch :  0 Step :  198 loss :  219.775619656
Epoch :  0 Step :  199 loss :  225.654294958
Epoch :  0 Step :  200 loss :  225.110165698
Epoch :  0 Step :  201 loss :  181.424323895
Epoch :  0 Step :  202 loss :  216.115963464
Epoch :  0 Step :  203 loss :  174.474877517
Epoch :  0 Step :  204 loss :  196.248302886
Epoch :  0 Step :  205 loss :  181.592650965
Epoch :  0 Step :  206 loss :  178.730505256
Epoch :  0

Epoch :  1 Step :  150 loss :  155.862485749
Epoch :  1 Step :  151 loss :  187.50820801
Epoch :  1 Step :  152 loss :  210.443295166
Epoch :  1 Step :  153 loss :  165.037545336
Epoch :  1 Step :  154 loss :  188.094728382
Epoch :  1 Step :  155 loss :  188.340358212
Epoch :  1 Step :  156 loss :  180.665518784
Epoch :  1 Step :  157 loss :  174.103899014
Epoch :  1 Step :  158 loss :  183.311480261
Epoch :  1 Step :  159 loss :  166.18661686
Epoch :  1 Step :  160 loss :  155.070011931
Epoch :  1 Step :  161 loss :  179.480057007
Epoch :  1 Step :  162 loss :  163.842344542
Epoch :  1 Step :  163 loss :  194.119801674
Epoch :  1 Step :  164 loss :  195.35177675
Epoch :  1 Step :  165 loss :  178.043739427
Epoch :  1 Step :  166 loss :  179.593945743
Epoch :  1 Step :  167 loss :  183.397798028
Epoch :  1 Step :  168 loss :  173.20139654
Epoch :  1 Step :  169 loss :  171.139748992
Epoch :  1 Step :  170 loss :  187.1219569
Epoch :  1 Step :  171 loss :  182.692521173
Epoch :  1 Step 

Epoch :  2 Step :  115 loss :  194.081723217
Epoch :  2 Step :  116 loss :  184.111055861
Epoch :  2 Step :  117 loss :  198.335145626
Epoch :  2 Step :  118 loss :  200.326937937
Epoch :  2 Step :  119 loss :  165.451902695
Epoch :  2 Step :  120 loss :  207.887417534
Epoch :  2 Step :  121 loss :  140.136174749
Epoch :  2 Step :  122 loss :  168.308030763
Epoch :  2 Step :  123 loss :  180.218909314
Epoch :  2 Step :  124 loss :  156.422797887
Epoch :  2 Step :  125 loss :  164.593220124
Epoch :  2 Step :  126 loss :  165.834189475
Epoch :  2 Step :  127 loss :  155.806977045
Epoch :  2 Step :  128 loss :  162.787564946
Epoch :  2 Step :  129 loss :  181.25900007
Epoch :  2 Step :  130 loss :  132.151480447
Epoch :  2 Step :  131 loss :  201.806485263
Epoch :  2 Step :  132 loss :  156.703318426
Epoch :  2 Step :  133 loss :  181.260930721
Epoch :  2 Step :  134 loss :  213.76371935
Epoch :  2 Step :  135 loss :  207.852970878
Epoch :  2 Step :  136 loss :  203.009890137
Epoch :  2 S

Epoch :  3 Step :  80 loss :  180.977780392
Epoch :  3 Step :  81 loss :  168.64485753
Epoch :  3 Step :  82 loss :  170.910113406
Epoch :  3 Step :  83 loss :  157.320975777
Epoch :  3 Step :  84 loss :  148.540188568
Epoch :  3 Step :  85 loss :  181.761814677
Epoch :  3 Step :  86 loss :  130.434487533
Epoch :  3 Step :  87 loss :  188.133051627
Epoch :  3 Step :  88 loss :  205.839357043
Epoch :  3 Step :  89 loss :  163.131907635
Epoch :  3 Step :  90 loss :  192.990573694
Epoch :  3 Step :  91 loss :  198.714155552
Epoch :  3 Step :  92 loss :  168.139491845
Epoch :  3 Step :  93 loss :  179.760983756
Epoch :  3 Step :  94 loss :  195.441143042
Epoch :  3 Step :  95 loss :  144.719060722
Epoch :  3 Step :  96 loss :  202.577532256
Epoch :  3 Step :  97 loss :  191.005568059
Epoch :  3 Step :  98 loss :  198.154593149
Epoch :  3 Step :  99 loss :  168.538669946
Epoch :  3 Step :  100 loss :  144.818730957
Epoch :  3 Step :  101 loss :  155.33573825
Epoch :  3 Step :  102 loss :  1

...

Epoch :  15 Step :  187 loss :  176.3754622
Epoch :  15 Step :  188 loss :  162.259708453
Epoch :  15 Step :  189 loss :  160.136062414
Epoch :  15 Step :  190 loss :  177.005831421
Epoch :  15 Step :  191 loss :  119.492058451
Epoch :  15 Step :  192 loss :  121.347046155
Epoch :  15 Step :  193 loss :  153.342108093
Epoch :  15 Step :  194 loss :  156.80974895
Epoch :  15 Step :  195 loss :  140.827046351
Epoch :  15 Step :  196 loss :  131.573559811
Epoch :  15 Step :  197 loss :  154.231915589
Epoch :  15 Step :  198 loss :  162.778607217
Epoch :  15 Step :  199 loss :  161.705023783
Epoch :  15 Step :  200 loss :  176.102783081
Epoch :  15 Step :  201 loss :  148.18507129
Epoch :  15 Step :  202 loss :  172.341163821
Epoch :  15 Step :  203 loss :  147.04658023
Epoch :  15 Step :  204 loss :  141.354379104
Epoch :  15 Step :  205 loss :  148.307497525
Epoch :  15 Step :  206 loss :  155.559079241
Epoch :  15 Step :  207 loss :  139.065949942
Epoch :  15 Step :  208 loss :  134.331

Epoch :  16 Step :  149 loss :  148.882328744
Epoch :  16 Step :  150 loss :  129.795355354
Epoch :  16 Step :  151 loss :  153.73392901
Epoch :  16 Step :  152 loss :  189.5274429
Epoch :  16 Step :  153 loss :  136.056847024
Epoch :  16 Step :  154 loss :  159.326486415
Epoch :  16 Step :  155 loss :  131.381611635
Epoch :  16 Step :  156 loss :  139.471978343
Epoch :  16 Step :  157 loss :  151.34543479
Epoch :  16 Step :  158 loss :  162.516314952
Epoch :  16 Step :  159 loss :  139.610681411
Epoch :  16 Step :  160 loss :  143.700342294
Epoch :  16 Step :  161 loss :  155.447589907
Epoch :  16 Step :  162 loss :  149.989874623
Epoch :  16 Step :  163 loss :  155.196304748
Epoch :  16 Step :  164 loss :  158.854203603
Epoch :  16 Step :  165 loss :  156.317170099
Epoch :  16 Step :  166 loss :  144.934856487
Epoch :  16 Step :  167 loss :  137.855434832
Epoch :  16 Step :  168 loss :  156.907059562
Epoch :  16 Step :  169 loss :  148.324612298
Epoch :  16 Step :  170 loss :  160.22

Epoch :  17 Step :  111 loss :  138.204290427
Epoch :  17 Step :  112 loss :  160.246433368
Epoch :  17 Step :  113 loss :  158.445789654
Epoch :  17 Step :  114 loss :  134.134315184
Epoch :  17 Step :  115 loss :  155.867427876
Epoch :  17 Step :  116 loss :  162.555518537
Epoch :  17 Step :  117 loss :  169.610284425
Epoch :  17 Step :  118 loss :  182.195605793
Epoch :  17 Step :  119 loss :  146.980846787
Epoch :  17 Step :  120 loss :  181.863745076
Epoch :  17 Step :  121 loss :  119.633030096
Epoch :  17 Step :  122 loss :  145.544968203
Epoch :  17 Step :  123 loss :  159.28205992
Epoch :  17 Step :  124 loss :  140.935647367
Epoch :  17 Step :  125 loss :  143.37323116
Epoch :  17 Step :  126 loss :  154.643540105
Epoch :  17 Step :  127 loss :  118.799416367
Epoch :  17 Step :  128 loss :  146.278744777
Epoch :  17 Step :  129 loss :  152.729333984
Epoch :  17 Step :  130 loss :  108.982203706
Epoch :  17 Step :  131 loss :  168.708532671
Epoch :  17 Step :  132 loss :  125.

Epoch :  18 Step :  72 loss :  190.242987503
Epoch :  18 Step :  73 loss :  157.39640619
Epoch :  18 Step :  74 loss :  149.228352474
Epoch :  18 Step :  75 loss :  141.175909712
Epoch :  18 Step :  76 loss :  176.548435431
Epoch :  18 Step :  77 loss :  164.205765024
Epoch :  18 Step :  78 loss :  125.430610808
Epoch :  18 Step :  79 loss :  164.411844139
Epoch :  18 Step :  80 loss :  154.933869745
Epoch :  18 Step :  81 loss :  146.028379484
Epoch :  18 Step :  82 loss :  154.891789066
Epoch :  18 Step :  83 loss :  131.242178664
Epoch :  18 Step :  84 loss :  132.488025571
Epoch :  18 Step :  85 loss :  159.409809553
Epoch :  18 Step :  86 loss :  115.414403004
Epoch :  18 Step :  87 loss :  161.174665452
Epoch :  18 Step :  88 loss :  172.275001739
Epoch :  18 Step :  89 loss :  132.733814795
Epoch :  18 Step :  90 loss :  167.00572281
Epoch :  18 Step :  91 loss :  169.251446971
Epoch :  18 Step :  92 loss :  139.203080873
Epoch :  18 Step :  93 loss :  160.516057264
Epoch :  18 

Epoch :  19 Step :  33 loss :  149.961075007
Epoch :  19 Step :  34 loss :  153.261933921
Epoch :  19 Step :  35 loss :  189.508616956
Epoch :  19 Step :  36 loss :  137.636288376
Epoch :  19 Step :  37 loss :  160.545258654
Epoch :  19 Step :  38 loss :  162.044685888
Epoch :  19 Step :  39 loss :  179.158196486
Epoch :  19 Step :  40 loss :  131.882653151
Epoch :  19 Step :  41 loss :  161.104327027
Epoch :  19 Step :  42 loss :  139.75461013
Epoch :  19 Step :  43 loss :  136.614306592
Epoch :  19 Step :  44 loss :  145.76267179
Epoch :  19 Step :  45 loss :  159.208887518
Epoch :  19 Step :  46 loss :  168.459568508
Epoch :  19 Step :  47 loss :  171.105087136
Epoch :  19 Step :  48 loss :  171.146539611
Epoch :  19 Step :  49 loss :  163.028907505
Epoch :  19 Step :  50 loss :  135.889551815
Epoch :  19 Step :  51 loss :  120.074463247
Epoch :  19 Step :  52 loss :  145.786581614
Epoch :  19 Step :  53 loss :  142.812463051
Epoch :  19 Step :  54 loss :  161.885015778
Epoch :  19 

In [129]:
(len(validating_data)-np.count_nonzero(model.predict(validating_data)-validating_data_class))/5000.0

0.8048

In [130]:
dumpModel(model)

In [91]:
testing_data=sk.preprocessing.normalize(test.iloc[:,1:785].copy().as_matrix())
#testing_data=clf.transform(testing_data)

In [92]:
test_results=model.predict(testing_data)

In [93]:
tmp=[i for i in range(10000)]

In [94]:
csv_result=np.array(zip(tmp,test_results))

In [95]:
print csv_result

[[   0    6]
 [   1    4]
 [   2    2]
 ..., 
 [9997    3]
 [9998    7]
 [9999    6]]


In [96]:
np.savetxt('./res_LATEST.csv',csv_result,delimiter=',',fmt='%d')